In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Learning goals
After today's lesson you should be able to:
- Use cross-validation
- Find the best model for classification and regression problems based on tuning hyperparameters and calculating performance scores

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

from pysal.lib import weights


## 0.1 Import the data

For this exercise, let's use the San Diego AirBnB data set again. As a reminder: This dataset contains house intrinsic characteristics, both continuous (number of beds as in `beds`) and categorical (type of renting or, in Airbnb jargon, property group as in the series of `pg_X` binary variables), but also variables that explicitly refer to the location and spatial configuration of the dataset (e.g., distance to Balboa Park, `d2balboa` or neighborhood id, `neighborhood_cleansed`).


Our aim here is to predict the **`log_price`** (Regression) of an AirBnB. 

In [ ]:
db = gpd.read_file("https://www.dropbox.com/s/zkucu7jf1xug869/regression_db.geojson?dl=1")


In [ ]:
db.info()

In [ ]:
db.head()

Again, notice here that we have: 
- **Discrete variables** (number of bedrooms, beds, baths)
- **Dummy variables** (whether there is a pool, whether near the coast, room type)

# 1. `log_price`
Let's start off with predicting the price of the Airbnb. 


In [ ]:
y = db['log_price']

## We'll make our X, independent variables, the "kitchen sink" of all of our other variables for now. 
## I'm using all the variables we have available with the exception of `neighborhood`, which we have to turn into dummy variables in a second. 

X = db[['accommodates', 'bathrooms', 'bedrooms', 'beds', 'pool',
       'd2balboa', 'coastal', 'pg_Apartment',
       'pg_Condominium', 'pg_House', 'pg_Other', 'pg_Townhouse',
       'rt_Entire_home/apt', 'rt_Private_room', 'rt_Shared_room']]

## 1.1 Feature engineering
Feature engineering is the process of creating new variables from the ones you already have. Common feature engineering tasks include:
- Creating dummy variables from categorical variables
- Creating interaction terms between variables
- Creating polynomial terms from variables
- Creating log or square root terms from variables
- Creating lagged variables from time series data or lagged spatial variables 


In [ ]:
neighborhood_dummies = pd.get_dummies(db['neighborhood'])

In [ ]:
neighborhood_dummies.head()

In [ ]:
## Here, I want to concatenate my X and neighborhood_dummies into one dataframe.
## I need to tell pd.concat() to either add new columns (axis=1) or add new rows (axis=0).
## The default is axis=0, ie new rows, so I need to specify axis=1.
X = pd.concat([X, neighborhood_dummies],axis=1)

In [ ]:
X.head()

Let's also create a new column that is the KNN spatial lag for the 'neighborhood context'. Here, I'm going to use the columns: 
- `pool`, which is a binary (0,1) variable for whether the airbnb has a pool
- `pg_House` which is a binary (0,1) variable for whether the airbnb is a house

I will choose K=20, to give me the 20 closest neighboring Airbnbs. My spatial lag should be a number between 0 and 20 to estimate, of the 20 closest Airbnb, how many have pools and how many are other housees.

In [ ]:
knn = weights.KNN.from_dataframe(db, k=20)

In [ ]:
pool_lag = weights.lag_spatial(knn, db['pool'])
house_lag = weights.lag_spatial(knn, db['pg_House'])

Add these to new features to our original data. 

In [ ]:
X['pool_lag'] = pool_lag
X['house_lag'] = house_lag


## 1.1 Create our Train-Test Split
We almost always start off with splitting our data into our **train** and **test** sets. 

In [ ]:
from sklearn.model_selection import train_test_split

## Let's use the default split for now, which is 75-25 train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,)


## 1.2 Predict the data
Here, let's use a decision tree regressor to predict the price. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## Use the permutation importance function from sklearn.inspection
from sklearn.inspection import permutation_importance


In [ ]:
model = RandomForestRegressor()

model.fit(X_train, y_train)

## 1.3 Cross-validation
Now we use the k-fold cross-validation method is run our model several times on different parts of the training data. 

In [ ]:
## The default scoring metric for Random Forest is R^2, so we can use cross_val_score() to get the R^2 for each fold.
scores = cross_val_score(model, X_train, y_train, cv=5)
scores

As you can see, there is some variation here. 

In [ ]:
scores.mean()

## 1.4 Test score 
Let's see how well our model does on the test data. 

In [ ]:
# evaluate the model on the second set of data
ypred_rf = model.predict(X_test)
print("R^2 is:", r2_score(y_test, ypred_rf))
print("Mean absolute error is:", mean_absolute_error(y_test, ypred_rf))
print("Mean squared error is:", mean_squared_error(y_test, ypred_rf))

## 1.5 Feature Importance for the Test Set
Note: You can also estimate the importance for the training set, but generally we're interested in performance for new data (test set). 

### 1.5.1 Impurity based feature importance

In [ ]:
importances = model.feature_importances_
std = np.std([model.feature_importances_ for tree in model.estimators_], axis=0)


In [ ]:
importances

In [ ]:
forest_importances = pd.Series(importances, index=X.columns)

fig, ax = plt.subplots(figsize=(13,6))
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()


### 1.5.2 Permutation based feature importance
This will take a few seconds because we have to reshuffle each column!

In [ ]:
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
forest_importances = pd.Series(result.importances_mean, index=X.columns)

In [ ]:
fig, ax = plt.subplots(figsize=(13,6))
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

## Q.1 Feature Importances 
Tune the hyperparameters on your model and measure the feature importances using the impurity and permutation based feature importances. Did your importances change?